# Corrección atmosférica en bucle

A continuación se detalla la modificación del código de Sam Murphy realizada para corregir en bucle todas las imágenes de una colección.

## Incluir paquetes necesarios e iniciar GEE

In [ ]:
# Descomentar la siguiente linea si condacolab no esta instalado
!pip install -q condacolab

**Importante**: Cuando se termina de ejecutar `condacolab.install()` el kernel se reinicia. Deberá volverse a ecutar el código y devolverá el mensaje "Everything looks OK!".

In [ ]:
# Importar comandos de conda
import condacolab
condacolab.install()

In [ ]:
# Instalar Py6S
!conda install conda-forge::py6s

In [ ]:
#@title Cargar modulo Atmospheric (funciones atmospheric.py)
"""
atmospheric.py, Sam Murphy (2016-10-26)

Atmospheric water vapour, ozone and AOT from GEE

Usage
H2O = Atmospheric.water(geom,date)
O3 = Atmospheric.ozone(geom,date)
AOT = Atmospheric.aerosol(geom,date)

"""

# El modulo ee sera importado mas adelante
# import ee

class Atmospheric():

  def round_date(date,xhour):
    """
    rounds a date of to the closest 'x' hours
    """
    y = date.get('year')
    m = date.get('month')
    d = date.get('day')
    H = date.get('hour')
    HH = H.divide(xhour).round().multiply(xhour)
    return date.fromYMD(y,m,d).advance(HH,'hour')

  def round_month(date):
    """
    round date to closest month
    """
    # start of THIS month
    m1 = date.fromYMD(date.get('year'),date.get('month'),ee.Number(1))

    # start of NEXT month
    m2 = m1.advance(1,'month')

    # difference from date
    d1 = ee.Number(date.difference(m1,'day')).abs()
    d2 = ee.Number(date.difference(m2,'day')).abs()

    # return closest start of month
    return ee.Date(ee.Algorithms.If(d2.gt(d1),m1,m2))



  def water(geom,date):
    """
    Water vapour column above target at time of image aquisition.

    (Kalnay et al., 1996, The NCEP/NCAR 40-Year Reanalysis Project. Bull.
    Amer. Meteor. Soc., 77, 437-471)
    """

    # Point geometry required
    centroid = geom.centroid()

    # H2O datetime is in 6 hour intervals
    H2O_date = Atmospheric.round_date(date,6)

    # filtered water collection
    water_ic = ee.ImageCollection('NCEP_RE/surface_wv').filterDate(H2O_date, H2O_date.advance(1,'month'))

    # water image
    water_img = ee.Image(water_ic.first())

    # water_vapour at target
    water = water_img.reduceRegion(reducer=ee.Reducer.mean(), geometry=centroid).get('pr_wtr')

    # convert to Py6S units (Google = kg/m^2, Py6S = g/cm^2)
    water_Py6S_units = ee.Number(water).divide(10)

    return water_Py6S_units



  def ozone(geom,date):
    """
    returns ozone measurement from merged TOMS/OMI dataset

    OR

    uses our fill value (which is mean value for that latlon and day-of-year)

    """

    # Point geometry required
    centroid = geom.centroid()

    def ozone_measurement(centroid,O3_date):

      # filtered ozone collection
      ozone_ic = ee.ImageCollection('TOMS/MERGED').filterDate(O3_date, O3_date.advance(1,'month'))

      # ozone image
      ozone_img = ee.Image(ozone_ic.first())

      # ozone value IF TOMS/OMI image exists ELSE use fill value
      ozone = ee.Algorithms.If(ozone_img,\
      ozone_img.reduceRegion(reducer=ee.Reducer.mean(), geometry=centroid).get('ozone'),\
      ozone_fill(centroid,O3_date))

      return ozone

    def ozone_fill(centroid,O3_date):
      """
      Gets our ozone fill value (i.e. mean value for that doy and latlon)

      you can see it
      1) compared to LEDAPS: https://code.earthengine.google.com/8e62a5a66e4920e701813e43c0ecb83e
      2) as a video: https://www.youtube.com/watch?v=rgqwvMRVguI&feature=youtu.be

      """

      # ozone fills (i.e. one band per doy)
      ozone_fills = ee.ImageCollection('users/samsammurphy/public/ozone_fill').toList(366)

      # day of year index
      jan01 = ee.Date.fromYMD(O3_date.get('year'),1,1)
      doy_index = date.difference(jan01,'day').toInt()# (NB. index is one less than doy, so no need to +1)

      # day of year image
      fill_image = ee.Image(ozone_fills.get(doy_index))

      # return scalar fill value
      return fill_image.reduceRegion(reducer=ee.Reducer.mean(), geometry=centroid).get('ozone')

    # O3 datetime in 24 hour intervals
    O3_date = Atmospheric.round_date(date,24)

    # TOMS temporal gap
    TOMS_gap = ee.DateRange('1994-11-01','1996-08-01')

    # avoid TOMS gap entirely
    ozone = ee.Algorithms.If(TOMS_gap.contains(O3_date),ozone_fill(centroid,O3_date),ozone_measurement(centroid,O3_date))

    # fix other data gaps (e.g. spatial, missing images, etc..)
    ozone = ee.Algorithms.If(ozone,ozone,ozone_fill(centroid,O3_date))

    #convert to Py6S units
    ozone_Py6S_units = ee.Number(ozone).divide(1000)# (i.e. Dobson units are milli-atm-cm )

    return ozone_Py6S_units


  def aerosol(geom,date):
    """
    Aerosol Optical Thickness.

    try:
      MODIS Aerosol Product (monthly)
    except:
      fill value
    """

    def aerosol_fill(date):
      """
      MODIS AOT fill value for this month (i.e. no data gaps)
      """
      return ee.Image('users/samsammurphy/public/AOT_stack')\
               .select([ee.String('AOT_').cat(date.format('M'))])\
               .rename(['AOT_550'])


    def aerosol_this_month(date):
      """
      MODIS AOT original data product for this month (i.e. some data gaps)
      """
      # image for this month
      img =  ee.Image(\
                      ee.ImageCollection('MODIS/006/MOD08_M3')\
                        .filterDate(Atmospheric.round_month(date))\
                        .first()\
                     )

      # fill missing month (?)
      img = ee.Algorithms.If(img,\
                               # all good
                               img\
                               .select(['Aerosol_Optical_Depth_Land_Mean_Mean_550'])\
                               .divide(1000)\
                               .rename(['AOT_550']),\
                              # missing month
                                aerosol_fill(date))

      return img


    def get_AOT(AOT_band,geom):
      """
      AOT scalar value for target
      """
      return ee.Image(AOT_band).reduceRegion(reducer=ee.Reducer.mean(),\
                                 geometry=geom.centroid())\
                                .get('AOT_550')


    after_modis_start = date.difference(ee.Date('2000-03-01'),'month').gt(0)

    AOT_band = ee.Algorithms.If(after_modis_start, aerosol_this_month(date), aerosol_fill(date))

    AOT = get_AOT(AOT_band,geom)

    AOT = ee.Algorithms.If(AOT,AOT,get_AOT(aerosol_fill(date),geom))
    # i.e. check reduce region worked (else force fill value)

    return AOT

In [ ]:
# Importar modulos necesarios
import ee
from Py6S import *
import datetime
import math
import os
import sys

Iniciar la API de EarthEngine.

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize(project='id') # ID de un proyecto asociado con la cuenta de google

## Se definen las tres funciones creadas por Sam Murphy

In [ ]:
# 1- Extraer la respuesta espectral
def spectralResponseFunction(bandname):

  """
  Respuesta espectral
  de las bandas Sentinel 2
  """
  bandSelect = {
  'B1':PredefinedWavelengths.S2A_MSI_01,
  'B2':PredefinedWavelengths.S2A_MSI_02,
  'B3':PredefinedWavelengths.S2A_MSI_03,
  'B4':PredefinedWavelengths.S2A_MSI_04,
  'B5':PredefinedWavelengths.S2A_MSI_05,
  'B6':PredefinedWavelengths.S2A_MSI_06,
  'B7':PredefinedWavelengths.S2A_MSI_07,
  'B8':PredefinedWavelengths.S2A_MSI_08,
  'B8A':PredefinedWavelengths.S2A_MSI_8A,
  'B9':PredefinedWavelengths.S2A_MSI_09,
  'B10':PredefinedWavelengths.S2A_MSI_10,
  'B11':PredefinedWavelengths.S2A_MSI_11,
  'B12':PredefinedWavelengths.S2A_MSI_12,
  }
  return Wavelength(bandSelect[bandname])

# 2- Conversion de reflectancia a radiancia3
def toa_to_rad(bandname):
  """
  Reflectividad TOA a radiancia
  """
  # Calcular la irradiancia solar exoatmosferica
  ESUN = info['SOLAR_IRRADIANCE_'+bandname]
  solar_angle_correction = math.cos(math.radians(solar_z))
  # Distancia Tierra-Sol (doy)
  doy = scene_date.timetuple().tm_yday
  d = 1 - 0.01672 * math.cos(0.9856 * (doy-4))
  # http://physics.stackexchange.com/
  # questions/177949/earth-sun-distance-on-a-given-day-of-the-year
  # factor de conversion
  multiplier = ESUN*solar_angle_correction/(math.pi*d**2)
  # calculo de radiancia
  rad = toa.select(bandname).multiply(multiplier)
  return rad

# 3- Calculo de reflectividad BOA
def surface_reflectance(bandname):
  """
  Calculo de la reflectividad en superficie a traves de la
  radiancia del sensor (en funcion de la longitud de onda,
  especifica para cada banda)
  """
  # Extraer la respuesta espectral de la banda
  s.wavelength = spectralResponseFunction(bandname)
  # Ejecutar los objetos 6s (definidos en la funcion principal)
  s.run()
  # Extraer las incognitas atmosfericas
  Edir = s.outputs.direct_solar_irradiance # irradiancia solar directa
  Edif = s.outputs.diffuse_solar_irradiance # irradiancia solar difusa
  Lp = s.outputs.atmospheric_intrinsic_radiance # path radiance
  absorb = s.outputs.trans['global_gas'].upward # absorption transmissivity
  scatter = s.outputs.trans['total_scattering']\
  .upward # scattering transmissivity
  tau2 = absorb*scatter # total transmissivity

  # Nota: los s.outputs son calculados automaticamente a partir de los
  # objetos 6s definidos en la funcion de conversion principal, "conversion".
  # Transformar los valores de reflectividad TOA a radiancia
  rad = toa_to_rad(bandname)
  # despejar la ecuacion de transferencia radiativa
  ref = rad.subtract(Lp).multiply(math.pi).divide(tau2*(Edir+Edif))
  # Devuelve la reflectividad a BOA de una banda
  return ref

## Definir el AOI sobre el que corregir la imagen

In [ ]:
# Incluir el area de estudio:
# Poligono a partir del cual se filtra la coleccion,
# se recorta la imagen final y se calculan los parametros
# atmosfericos necesarios
geom = ee.Geometry.Polygon([[-0.9570796519011493,40.98197275411647],
[-0.5670650034636493,40.98197275411647],
[-0.5670650034636493,41.45919658393617],
[-0.9570796519011493,41.45919658393617],
[-0.9570796519011493,40.98197275411647]])

# Descomentar la siguiente linea si ee.Geometry.Polygon no funciona
# geom = ee.Geometry.Rectangle(-0.996, 41.508, -0.568, 40.992)
# Obtener las coordenadas de geom para recortar las imagenes de la coleccion
region = geom.buffer(1000).bounds().getInfo()['coordinates']

## Creacion de la función para corregir imágenes en bucle

In [ ]:
def conversion(img, assetID):
  # Incorporar la fecha de la imagen
  date = img.date()
  # Definir las variables globales:
  # Aquellas que pueden ser llamadas fuera del entorno de la funcion.
  global toa
  global info
  global scene_date
  global solar_z

  # calcular la reflectividad a TOA
  toa = img.divide(10000)

  # Escribir los metadatos de la imagen
  # Recopilar las propiedades
  info = img.getInfo()['properties']
  # Fecha: Python utiliza segundos, EE milisegundos
  scene_date = datetime.datetime\
  .utcfromtimestamp(info['system:time_start']/1000)
  # Angulo cenital solar
  solar_z = info['MEAN_SOLAR_ZENITH_ANGLE']
  # Valores sobre la composicion atmosferica
  # El codigo de las funciones se encuentra dentro
  # del repositorio de samsammurphy (gee-atmcorr-S2/bin/atmospheric.py)
  h2o = Atmospheric.water(geom,date).getInfo()
  o3 = Atmospheric.ozone(geom,date).getInfo()
  # Atmospheric Optical Thickness
  aot = Atmospheric.aerosol(geom,date).getInfo()
  # Altura de la superficie, a partir del MDE de la mision
  # Shuttle Radar Topography mission (STRM) en GEE
  SRTM = ee.Image('CGIAR/SRTM90_V4')

  # Calculo de la altura media del ´area de estudio (geom)
  alt = SRTM.reduceRegion(reducer = ee.Reducer.mean(),
  geometry = geom.centroid()).get('elevation').getInfo()
  # Transformar a km, medida utilizada por Py6s
  km = alt/1000

  """
  Inicio de los objetos 6s, columna vertebral de Py6s
  A partir de la clase 6s se definen los parametros
  requeridos por la funcion de transferencia radiativa
  Llamar a los objetos 6s
  """
  global s
  s = SixS()
  # Integrar los componentes atmosfericos
  s.atmos_profile = AtmosProfile.UserWaterAndOzone(h2o,o3)
  s.aero_profile = AeroProfile.Continental
  s.aot550 = aot
  # Calcular la geometria Earth-Sun-satellite
  s.geometry = Geometry.User()
  s.geometry.view_z = 0 # calculo asumiendo vision en NADIR
  s.geometry.solar_z = solar_z # angulo cenital solar
  s.geometry.month = scene_date.month # mes usado en la distancia Earth-Sun
  s.geometry.day = scene_date.day # dia usado en la distancia Earth-Sun
  s.altitudes\
  .set_sensor_satellite_level() # Altitud del sensor
  s.altitudes\
  .set_target_custom_altitude(km) # Altitud de la superficie

  # Aplicar la conversion a cada banda de la imagen
  # 1. Generar el objeto (imagen) a exportar
  output = img.select('QA60')
  # 2. Bucle de correccion: aplica la funcion de correccion a las bandas
  # de la lista
  for band in ['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']:
    # Corregir la banda e incluirla en la imagen a exportar
    output = output.addBands(surface_reflectance(band))

  # Exportar la imagen a una carpeta en GEE
  # 1. Definir parametros de la imagen a exportar
  dateString = scene_date.strftime("%Y-%m-%d")
  ref = output.set({'satellite':'Sentinel 2',
    'fileID':info['system:index'],
    'date':dateString,
    'aerosol_optical_thickness':aot,
    'water_vapour':h2o,
    'ozone':o3})

  # Crear el nomrbe de la imagen a exportar
  imageID = assetID + 'S2SR_'+dateString
  # 2. Opciones de la imagen a exportar
  export = ee.batch.Export.image.toAsset(\
    image=ref,
    description='sentinel2_atmcorr_export',
    assetId = imageID,
    region = region,
    crs = 'EPSG:4326',
    scale = 20)

  # 3. Iniciar el task
  export.start()
  return print("Imagen "+assetID+" en proceso de descarga.")

# Final de la funcion de conversion en bucle

## Aplicar la funcion de conversion en bucle a una coleccion GEE

Primero se define la colección y despues se aplica la conversión dentro de un bucle a todas las imágenes que contiene.

In [ ]:
# Definir coleccion GEE
S2 = ee.ImageCollection('COPERNICUS/S2')\
  .filterBounds(geom)\
  .filterDate('2015-10-01','2017-04-30')\
  .filterMetadata('MGRS_TILE', 'equals', '30TXL')\
  .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 20)\
  .sort('system:time_start')\
  .distinct('system:time_start')

# Definir en una lista las imagenes a filtrar
features = S2.getInfo()['features']

# Definir la carpeta de destino (dentro de GEE)
assetID = 'users/iranzocristian/6s_test_2023/'

"""
CORRECCION DE LA COLECCION AUTOMATICAMENTE (bucle for)
1. Recorre cada imagen de la lista anterior
2. Obtiene su id
3. Se llama a la imagen de la coleccion GEE con el id anterior
4. Se aplica la funcion de conversion principal
"""
for i in features:
  id = i['id']
  conversion(ee.Image(id), assetID)
# Final del Script

## Cargar la coleccion de imagenes corregidas

En la API de JavaScript (code editor) debería incluirse el siguiente código:

```javascript
var assetList = ee.data.listAssets("users/iranzocristian/6s_test_2023")['assets']
                    .map(function(d) { return d.name })
var collection = ee.ImageCollection(assetList)
```

In [ ]:
assetList = ee.data.listAssets("users/iranzocristian/6s_test_2023")['assets']
assetNames = [i['name'] for i in assetList]
collection = ee.ImageCollection(assetNames)
print('Has cargado una colección con ' + str(collection.size().getInfo()) + ' imgs corregidas.')